In [ ]:
import rasterio
import geopandas as gpd
from rasterio.features import rasterize
import numpy as np

def create_roof_mask(image_path, geojson_path, output_path):
    # load satellite image metadata
    with rasterio.open(image_path) as src:
        crs = src.crs
        transform = src.transform
        width = src.width
        height = src.height

    # load and reproject GeoJSON
    gdf = gpd.read_file(geojson_path).to_crs(crs)
    
    # create class mapping
    materials = gdf['roof_material'].unique()
    class_mapping = {mat: i+1 for i, mat in enumerate(materials)}

    print(class_mapping)
    

    shapes = [(geom, class_mapping[roof]) 
              for geom, roof in zip(gdf.geometry, gdf["roof_material"])]

    mask = rasterize(
        shapes=shapes,
        out_shape=(height, width),
        transform=transform,
        fill=0,  # background value
        dtype=np.uint8,
        all_touched=True
    )

    # save mask
    with rasterio.open(output_path, 'w', 
                       driver='GTiff',
                       height=height,
                       width=width,
                       count=1,
                       dtype=np.uint8,
                       crs=crs,
                       transform=transform) as dst:
        dst.write(mask, 1)

satellite_image_file_path = '' #iterate through satellite images here!
create_roof_mask(satellite_image_file_path, "filtered_categories_buildings.geojson", "roof_mask.tif")